In [89]:
import numpy as np;
from math import sin, cos, sqrt, atan2, radians,inf;
import random; 

class Autocar:   
    def __init__(self,AC_location = 0,state_of_charge = -1,min_lat = 52,
                 min_long = 12.9,capacity = 75, powerpdist = 14):
        #map constraint
        self.AC_location = np.random.random(2) + (min_lat,min_long)
        self.min_lat = min_lat;
        self.min_long = min_long;
        # car representet as random distribution funktion --> random langitude and latidude
        #AC_long = np.random.random(1);
        #AC_lat = np.random.random(1);
        # alternative: random function generates random vector that represent a car with only 20% of charge
        # random number amount between 0 and 1
        # car needs parameters
        self.capacity  = capacity; # example for Tessla S modal, BMWi3 ; smart60kW fortwo 17,6kWh
        #possible_distance = 400 # Tessla s Modal; 260km BMWi3 (daily distance)
        #Stromverbrauch in kWh/100 km: 14,6 - 14,0
        self.powerpdist = powerpdist;#power per distance --> kWh per 100km
        #powtodist = powerpdist/100; #power [kWh] need for 1km; theoretically something that might be measured in the car andoptimized by the car itself
        self.powerkm = powerpdist/100
        if state_of_charge < 0:    
            self.state_of_charge = round(random.uniform(0.2,1), 3); # state of battery between 0 and 1 --> if it is smaller 0.2 it will check where it can charge its battery
        else:
            self.state_of_charge = state_of_charge;
            
    def show_attributes(self):
        print('location:',self.AC_location,' (min_lat:', self.min_lat,'min_long:', self.min_long,')\ncapacity [kWh]:',
              self.capacity,'\npower consumption per km:', self.powerkm,'\nstate of charge [% of capacity]:',self.state_of_charge*100 )

    def find_distance_km(self, lat1,lng1,lat2,lng2):
        EARTH_RADIUS = 6373.0
        lat1 = radians(lat1)
        lng1 = radians(lng1)
        lat2 = radians(lat2)
        lng2 = radians(lng2)
        
        dlon = lng2 - lng1
        dlat = lat2 - lat1
        
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        
        return EARTH_RADIUS * c 
    
class Grid:   
    def __init__(self, grid_location = 0, total_charge_needed_at_grid=0, dist=0, price=0, name = "default"):
        #map constraint
        self.grid_location = np.random.random(2) + (min_lat,min_long)
        self.total_charge_needed_at_grid = total_charge_needed_at_grid
        self.dist = dist
        self.price = price
        self.name = name


In [95]:
import random
import pandas as pd
from pulp import * 

min_lat = 52
min_long = 12.9


def create_vehicles(num):
    acs = []
    for x in range(num):
        acs.append(Autocar(capacity = random.uniform(50,120), powerpdist = random.uniform(5, 20)))
    return acs

def create_grids(num):
    grids_loc = []
    for x in range(num):
        grids_loc.append(Grid(name="grid "))
    return grids_loc

def reachable_grids(ac, grids):
    final_grids = []
    for x in range(len(grids)):
        ### calculate total energy to grid considering distance
        grids[x].dist = ac.find_distance_km(grids[x].grid_location[0], grids[x].grid_location[1], ac.AC_location[0], ac.AC_location[1])
        c_to_grid = ac.powerkm*grids[x].dist
        ### consider only reachable grids
        if (c_to_grid <= (ac.state_of_charge*ac.capacity)):
            ### add the energy needed to reach grid to the total deficit
            grids[x].total_charge_needed_at_grid = c_to_grid+(ac.capacity-(ac.state_of_charge*ac.capacity))
            #energy_deficit.append(total_charge[0])
            grids[x].price = random.uniform(0.4,0.8)
            ### location, total energy needed at grid location, distance, price
            final_grids.append(grids[x])
    return final_grids

#############################
### linear Programming#######
#############################

def minimize_cost():

    prob = LpProblem("AC", LpMinimize)

### every x represents a boolean variable wether to pick a grid
### considering only 3 grids currently
    grid_1 = LpVariable("grid 1",0, 1, LpInteger)
    grid_2 = LpVariable("grid 2",0, 1, LpInteger)
    grid_3 = LpVariable("grid 3",0, 1, LpInteger)


### OBJECTIVE FUNCTION
    #for x in range(2):
    prob += (energy_deficit[0]*grids_price[0]*grid_1+ energy_deficit[1]*grids_price[1]*grid_2+ energy_deficit[2]*grids_price[2]*grid_3)


### CONSTRAINTS
# all binary decision variables should sum up to 1
    prob += grid_1+grid_2+grid_3 == 1

    prob.writeLP("AC.lp")
    prob.solve()

#print("grid 1: %d , grid 2: %d , grid 3: %d " % (value(grid_1), value(grid_2), value(grid_3)))
    val = [grid_1, grid_2, grid_3]

    for x in val:
        if(value(x)==1):
        #print("all grid locations:", grids_loc_final[0:3])
            print("all grid prices:", grids_price[0:3])
            print("total energy deficit at grid:", energy_deficit[0:3])
            print("Lowest cost at..", x)
            print("Price: ", value(prob.objective))

def minimize_distance(grids):

    prob = LpProblem("AC", LpMinimize)

### every x represents a boolean variable wether to pick a grid
### considering only 3 grids currently
    grid_1 = LpVariable("grid 1",0, 1, LpInteger)
    grid_2 = LpVariable("grid 2",0, 1, LpInteger)
    grid_3 = LpVariable("grid 3",0, 1, LpInteger)

    grids_dist = [grids[0].dist,grids[1].dist,grids[2].dist]
    #print(grids_dist)
### OBJECTIVE FUNCTION
    #for x in range(2):
    prob += grids_dist[0]*grid_1+grids_dist[1]*grid_2+grids_dist[2]*grid_3


### CONSTRAINTS
# all binary decision variables should sum up to 1
    prob += grid_1+grid_2+grid_3 == 1

    prob.writeLP("AC.lp")
    prob.solve()

#print("grid 1: %d , grid 2: %d , grid 3: %d " % (value(grid_1), value(grid_2), value(grid_3)))
    val = [grid_1, grid_2, grid_3]

    for x in val:
        if(value(x)==1):
        #print("all grid locations:", grids_loc_final[0:3])
            print("all grid distances:", grids_dist[0:3])
            #print("Lowest distance to..", grids[x].name)
            print("Lowest distance: ", value(prob.objective))
            print("Car fully charged! 100% battery!")

def drive_vehicle(ac, grids):
        # assuming v = 50km/h
        soc = random.uniform(0.4,1)
        code = 0
        ### available energy
        powerstate = ac.state_of_charge*ac.capacity
        ### needed energy
        consumption = 0.9*ac.powerpdist
        if (powerstate-consumption > 0):
            ### available energy - needed energy
            powerupdate = powerstate-consumption
            ### percentage battery left (new state of charge)
            soc_update = round((powerupdate/ac.capacity), 2)
            if (soc_update <= 0):
                print(soc_update)
                print("Battery below 1%.. Randomly resetting SOC to ", soc)
            if (soc_update <= 0.2):
                print("Searching for nearest charging station..")
                minimize_distance(grids)
                soc = 1.0
                ### code for update within reach
                code = 1
            else:
                soc = soc_update
                print("Current state of charge: ", soc_update)
        else:
            print("Car out of power.. Randomly resetting SOC to ", soc)
        ac.state_of_charge = soc
        return code
        #print(soc_update)
        #print("driving... battery status:", acs[x].state_of_charge[0])


In [98]:
from apscheduler.schedulers.background import BackgroundScheduler as scheduler
import apscheduler

scheduler = scheduler()
scheduler.start()

### Create vehicles
vehicles = create_vehicles(3)
### Create grids
grids = create_grids(10)
### choose car 
ac = vehicles[0]
### Find grids which can be reached with current state of charge
grids_within_reach = reachable_grids(ac, grids)

def listener(event):
    if(event.code == 4096):
        if event.retval==1:
            ac.AC_location = np.random.random(2) + (min_lat,min_long)
            grids_within_reach = reachable_grids(ac, grids)

        #ac.AC_location = np.random.random(2) + (min_lat,min_long)
        #grids_within_reach = reachable_grids(ac, grids)
        #print(ac.AC_location)
        #print(grids_within_reach[0:3])


scheduler.add_job(drive_vehicle, 'interval', args=[ac,grids_within_reach], seconds=1)
scheduler.add_listener(listener)

Current state of charge:  0.82
Current state of charge:  0.75
Current state of charge:  0.68
Current state of charge:  0.61
Current state of charge:  0.54
Current state of charge:  0.47
Current state of charge:  0.4
Current state of charge:  0.33
Current state of charge:  0.26
Searching for nearest charging station..
all grid distances: [16.524851984076644, 46.91880766299857, 13.702474066743559]
Lowest distance:  13.702474066743559
Car fully charged! 100% battery!
Current state of charge:  0.93
Current state of charge:  0.86
Current state of charge:  0.79
Current state of charge:  0.72
Current state of charge:  0.65
Current state of charge:  0.58
Current state of charge:  0.51
Current state of charge:  0.44
Current state of charge:  0.37
Current state of charge:  0.3
Current state of charge:  0.23
Searching for nearest charging station..
all grid distances: [54.31416770795786, 38.35528976464137, 35.33357593677798]
Lowest distance:  35.33357593677798
Car fully charged! 100% battery!
Cur

In [99]:
scheduler.shutdown(wait=False)

Current state of charge:  0.3
